<a href="https://colab.research.google.com/github/RoetGer/decisions-under-uncertainty/blob/main/data_science_and_stochastic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cvxpy
!pip install cvxstoc

     |████████████████████████████████| 389kB 7.6MB/s 
  Created wheel for pymc: filename=pymc-2.3.8-cp37-cp37m-linux_x86_64.whl size=1352885 sha256=bf925cee21b11d94b2ccd92ed200423b2b569ee21251f0b912e7ea029e01c3fe
  Stored in directory: /root/.cache/pip/wheels/0b/a8/e7/8f3ba91a39294d538a92db052fd1fcba1fca74a58c8b022026
Successfully built pymc


# Data Science and Stochastic Programming

In this notebook we explore, how stochastic programming can be used to incorporate uncertainty stemming from data science models into our decision making process.

Let us start by introducing cvxstoc, a Python package for solving stochastic convex optimization problems.

In [4]:
import cvxpy
import cvxstoc
import numpy as np
import pymc

from cvxstoc import NormalRandomVariable, expectation, prob
from cvxpy import Maximize, Problem
from cvxpy.expressions.variable import Variable

In [5]:
# Samples to be taken
num_samples = 100

# Create problem data.
n = 5
mu = np.zeros(n)
Sigma = 0.1*np.eye(n)
returns = NormalRandomVariable(mu, Sigma)
alpha = -0.5
beta = 0.05

# Create the stochastic optimization problem.
weights = Variable(n)
probl = Problem(
    Maximize(expectation(weights.T*returns, num_samples=num_samples)),
    [
      cvxpy.max(weights) <= 0.3,
      weights >= 0, 
      weights.T*np.ones(n) == 1,
      prob(weights.T*returns <= alpha, num_samples=num_samples) <= beta
    ]
)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


What we are trying to solve here is a simplified portfolio allocation problem, where the goal is to find a weight vector which maximizes the return under some constraints. 

The main differences to a more classical approach is that we are not working with a fixed vector of returns, but we assume that the returns are following a Gaussian distribution (with mean mu and covariance Sigma).

A consequence of this choice is that we are not merely trying to maximize the weighted sum of the returns (= weights.T*returns), but an expectation of this weighted sum with respect to the uncertain returns.

Moreover, while the first three constraints are rather standard (none of the portfolio positions should exceed 30% of the overall portfolio, the weights should be non-negative, and the combined weights add up to one), the last one is different from a deterministic optimization problem. The last constraint restricts the probability of the optimal portfolio to exceed a loss of 50% to 5%, i.e. for 100 samples of the return vector, we would only expect to have 5 times a loss higher than 50% with the optimized weights.

In [6]:
probl.solve()

print(probl.status)
print("Optimal value:", probl.value)
print("Optimal weights:", weights.value)

optimal
Optimal value: 0.021495551507213138
Optimal weights: [2.99999999e-01 1.00000001e-01 1.04553238e-11 3.00000000e-01
 3.00000000e-01]


While it is fairly straightforward to see how this approach can be integrated with a data science solution (i.e. the data science model provides mean and covariance estimates for the Gaussian distribution), it is rather limited in its usage with a model.

For example, if we are using a Bayesian model to obtain posterior predictive samples, utilize dropout with a deep learning model to generate samples, or simply not use one of the distributions currently supported by cvxstocm, we would not be able to solve the resulting optimization problem.

In order to simplify the work with more complex distribution, we have developed the following function

In [43]:
import numpy as np
import pymc
from cvxstoc import RandomVariable


def EmpiricalRandomVariable(name, 
                            samples,
                            mean,
                            interpolate=False, 
                            lower=-np.inf, 
                            upper=np.inf):
    '''
    Create a pymc node whose distribution comes either from a 
    kernel smoothing density estimate or via boostrapping from 
    the provided samples.
    '''
    
    if interpolate:
      rv_pymc = pymc.stochastic_from_data(
          name=rv_name, 
          data=samples, 
          lower=lower, 
          upper=upper)
    else:
        nobs = samples.shape[0]

        def logp(value):
            return -np.log(nobs)

        def random():
            ridx = np.random.randint(low=0, high=nobs, size=1)
            return np.squeeze(samples[ridx])

        value = random() 
        dtype = type(value)
    
        rv_pymc = pymc.Stochastic(
            logp = logp,
            doc = "A node which bootstrap samples from the provided dataset",
            name = name,
            parents = {},
            random = random,
            trace = True,
            dtype = dtype)
    
    metadata = {"mu": mean}
    
    return RandomVariable(rv=rv_pymc, metadata=metadata)

In [44]:
# Samples to be taken
num_samples = 100

# Create problem data.
n = 5
mu = np.zeros(n)
Sigma = 0.1*np.eye(n)
returns = EmpiricalRandomVariable("EmpiricalRV", 
                                  NormalRandomVariable(mu, Sigma).sample(100),
                                  mean = mu,
                                  interpolate=False)
alpha = -0.5
beta = 0.05

# Create the stochastic optimization problem.
weights = Variable(n)
probl = Problem(
    Maximize(expectation(weights.T*returns, num_samples=num_samples)),
    [
      cvxpy.max(weights) <= 0.3,
      weights >= 0, 
      weights.T*np.ones(n) == 1,
      prob(weights.T*returns <= alpha, num_samples=num_samples) <= beta
    ]
)

probl.solve()

print(probl.status)
print("Optimal value:", probl.value)
print("Optimal weights:", weights.value)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


optimal
Optimal value: 0.012308462355088107
Optimal weights: [3.0000000e-01 1.0000000e-01 1.0320114e-11 3.0000000e-01 3.0000000e-01]


We can extend the example by optimizing a mean-variance problem, but in addition to the uncertain returns, we also assume that the covariance matrix is uncertain.

---



In [9]:
??pymc.WishartCov

In [11]:
n

5

In [19]:
pymc.WishartCov("cov-mat", 5, np.eye(n)).random()

matrix([[ 3.32181198,  2.15145475, -0.28322805, -0.4729046 , -0.17836496],
        [ 2.15145475,  1.77319592,  0.85076045,  0.63573653, -0.04151246],
        [-0.28322805,  0.85076045,  6.17560499,  2.29282081,  3.40618157],
        [-0.4729046 ,  0.63573653,  2.29282081,  5.53302866, -2.27847308],
        [-0.17836496, -0.04151246,  3.40618157, -2.27847308,  4.91719708]])

In [35]:
type(np.random.randn(3, 4))

numpy.ndarray

In [36]:
type(np.array(pymc.WishartCov("cov-mat", 5, np.eye(n)).random()))

numpy.ndarray

In [73]:
cov_samples = []

for i in range(2):
  sig = np.random.randn(n, n)
  sig = sig.T.dot(sig)
  cov_samples.append(sig)

cov_samples = np.stack(cov_samples, axis=0)

In [50]:
cov_samples = [
    np.eye(n) + np.array(pymc.WishartCov("cov-mat", 5, Sigma).random()) 
      for _ in range(10)
]
cov_samples = np.stack(cov_samples, axis=0)

In [48]:
np.squeeze(cov_samples[1].shape)

array([5, 5])

TODO: There seems to be some issue with quadform when it comes to stochastic programs.

In [77]:
from cvxpy import Minimize

# Samples to be taken
num_samples = 100

# Create problem data.
n = 5
mu = np.zeros(n)
Sigma = 0.1*np.eye(n)
returns = EmpiricalRandomVariable("EmpiricalReturns", 
                                  NormalRandomVariable(mu, Sigma).sample(100),
                                  mean = mu,
                                  interpolate=False)
cov_mats = EmpiricalRandomVariable("CovMats",
                                   cov_samples,
                                   mean = Sigma,
                                   interpolate=False)
cov_mats = cov_samples[0] + np.eye(n)
alpha = -0.5
beta = 0.05

# Create the stochastic optimization problem.
weights = Variable(n)
probl = Problem(
    Maximize(expectation(
        weights.T*returns - cvxpy.quad_form(weights, cov_mats),
        num_samples=num_samples)),
    [
      cvxpy.max(weights) <= 0.3,
      weights >= 0, 
      weights.T*np.ones(n) == 1,
      prob(weights.T*returns <= alpha, num_samples=num_samples) <= beta
    ]
)

probl.solve(verbose=True)

print(probl.status)
print("Optimal value:", probl.value)
print("Optimal weights:", weights.value)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 107, constraints m = 214
          nnz(P) + nnz(A) = 938
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.3176e-04   1.00e+00   3.58e+02   1.00e-01   5.46e-04s
 200   6.5836e-01   6.22e-02   5.76e-04   5.99e+00   1.94e-03s
 400   6.5851e-01   6.08e-02   5.62e-04   5.9

SolverError: ignored

In [61]:
np.linalg.eig(cov_mats)

(array([7.10404969, 4.56014072, 2.61043479, 0.00844693, 0.02005087]),
 array([[-9.83828349e-02,  6.81913286e-01,  5.57104073e-02,
          4.83932667e-01,  5.36675519e-01],
        [ 3.38493596e-01, -4.09862823e-01,  6.99038116e-01,
          3.70354433e-02,  4.76873821e-01],
        [-9.27422518e-01, -1.42683572e-01,  3.13265772e-01,
         -1.18552851e-01,  8.56659176e-02],
        [ 1.25019253e-01,  5.88055712e-01,  4.77614592e-01,
         -5.95121327e-01, -2.37224733e-01],
        [ 7.50733306e-04, -2.89588494e-02, -4.26603939e-01,
         -6.29454825e-01,  6.48811624e-01]]))